In [31]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import permutations
import copy
import timeit



In [2]:
def load_original_data():
    f = open("voters.txt")
    txt = f.read()
    preferences = [val.split(': ')[-1] for val in txt.split('\n')]
    counts = [float(val.split(': ')[0]) for val in txt.split('\n')]
    preferences_to_counts = {preference: count for preference, count in zip(preferences, counts)}
    preferences_to_counts = defaultdict(int, preferences_to_counts)
    
#     deal with ties: ex 1, {8, 10} -> 1, 8 & 1, 10
    new_preferece_counts = []
    preferences_with_ties = []
    for preference, count in zip(preferences, counts):
        ties = re.findall("{\d+,\d+}", preference)
        n_ties = len(ties)
        if(n_ties>0):
            print(preference)
            preferences_with_ties.append(preference)
    #       we expect only one tie for each preference of alternatives
            tie = re.sub("[{}]", "", ties[0]).split(',')
            for t in tie:
                new_preferece_counts.append((re.sub(re.escape(ties[0]), t, preference), count/2))

    for preference, extra_counts in new_preferece_counts:
        preferences_to_counts[preference] += extra_counts
    for preference in preferences_with_ties:
        del preferences_to_counts[preference]

    preferences_to_counts
        
    preferences = [preference.split(',') for preference in list(preferences_to_counts.keys())]
    counts = list(preferences_to_counts.values())
    
    df =  pd.DataFrame([counts, preferences]).T
    df.rename(columns={0: 'n', 1: 'preference'}, inplace=True)
    
    print(f"Total voters: {df.n.sum()}")
    return df

In [3]:
df = load_original_data()

3,4,5,6,7,8,9,{1,10},2
2,4,5,7,{9,10},3,6,1,8
1,2,6,5,3,7,8,9,{10,11}
4,2,5,3,1,6,7,8,{9,10}
9,3,2,5,{1,4},6,7,8
2,8,3,4,{1,5},6,9,7
3,5,1,{7,9},8,6,4,2
3,1,2,5,6,8,{9,10}
2,4,1,{5,8},6,9
Total voters: 2477.0


In [4]:
df.tail(18)

,n,preference
1751,0.5,"[3, 4, 5, 6, 7, 8, 9, 1, 2]"
1752,0.5,"[3, 4, 5, 6, 7, 8, 9, 10, 2]"
1753,0.5,"[2, 4, 5, 7, 9, 3, 6, 1, 8]"
1754,0.5,"[2, 4, 5, 7, 10, 3, 6, 1, 8]"
1755,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 10]"
1756,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 11]"
1757,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 9]"
1758,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 10]"
1759,0.5,"[9, 3, 2, 5, 1, 6, 7, 8]"
1760,0.5,"[9, 3, 2, 5, 4, 6, 7, 8]"


In [5]:
alternatives = [str(i) for i in range(1, 12)]
alternatives

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [6]:
# number of total voters
df.n.sum()

2477.0

In [7]:
# df = df.copy()
df = df.sample(frac =.10, random_state=42)

In [8]:
def get_p_scores(df, alternatives):
    scores = {}
    for a in alternatives:
        scores[a] = 0
    for index, row in df.iterrows():
        if(len(row.preference) > 0):
            scores[row.preference[0]] += int(row.n)
    return scores

def get_score_alternatives(scores, mode):
    mode_score = min(scores, key=scores.get)
    if(mode == 'max'):
        mode_score = max(scores, key=scores.get)
    mode_score_alternatives = [i for i, val in scores.items() if val == scores[mode_score]]
    return mode_score_alternatives

def remove_alt(row, alternatives):
    preference = copy.deepcopy(row.preference)
    for alt in alternatives:
        try:
            preference.remove(alt)
        except:
            pass
    return preference
def remove_alternatives(df, alternatives):
    # df.preference = df.apply(lambda x: list(set(x.preference).difference(set(alternatives))), axis=1)
    df.preference = df.apply(lambda x: remove_alt(x, alternatives=alternatives), axis=1)
    return df

def STV(df, alternatives, verbose=1):
    votes_df = copy.deepcopy(df)
    # votes_df = df.copy(deep=True)
    scores = get_p_scores(votes_df, alternatives)
    while(len(alternatives)>0):
        scores = get_p_scores(votes_df, alternatives)
        min_p_alternatives = get_score_alternatives(scores, 'min')
        alternatives = list(set(alternatives).difference(set(min_p_alternatives)))
        if(len(alternatives)==0):
            if(verbose > 0):
                print("Reached a decision")
            break
        if(verbose > 0):
            print(scores)
            print('removing alternatives', min_p_alternatives)
        votes_df = remove_alternatives(votes_df, min_p_alternatives)
        if(verbose > 1):
            print(votes_df)
    return list(scores.keys())

In [9]:
winners = STV(df, alternatives)
winners

{'1': 26, '2': 33, '3': 56, '4': 21, '5': 27, '6': 6, '7': 9, '8': 42, '9': 2, '10': 0, '11': 0}
removing alternatives ['10', '11']
{'9': 2, '5': 27, '8': 42, '7': 9, '1': 26, '6': 6, '3': 56, '4': 21, '2': 33}
removing alternatives ['9']
{'5': 27, '8': 43, '7': 9, '1': 26, '6': 6, '3': 57, '4': 21, '2': 33}
removing alternatives ['6']
{'5': 27, '8': 45, '7': 10, '1': 27, '3': 58, '4': 22, '2': 33}
removing alternatives ['7']
{'5': 29, '8': 47, '1': 28, '3': 60, '4': 24, '2': 34}
removing alternatives ['4']
{'5': 31, '8': 51, '1': 32, '3': 63, '2': 45}
removing alternatives ['5']
{'1': 40, '2': 48, '3': 67, '8': 63}
removing alternatives ['1']
{'2': 63, '3': 75, '8': 78}
removing alternatives ['2']
{'3': 94, '8': 101}
removing alternatives ['3']
Reached a decision


['8']

In [10]:
df

,n,preference
974,1.0,"[8, 3, 4, 1, 5, 9, 6, 2]"
275,1.0,"[8, 3, 4, 1, 5, 6, 9, 2, 7]"
411,1.0,"[3, 8, 6, 7, 5, 9, 1, 4, 2]"
962,1.0,"[5, 1, 8, 4, 3, 6, 7, 9]"
518,1.0,"[5, 1, 2, 4, 7, 6, 8, 3, 9]"
...,...,...
239,1.0,"[2, 1, 8, 4, 9, 5, 3, 7, 6]"
168,2.0,"[2, 5, 9, 4]"
914,1.0,"[5, 1, 7, 4, 3, 8, 9, 2]"
1173,1.0,"[1, 5, 2, 8, 3, 6]"


In [11]:
def consider_alternative(x):
    w_found = 0
    winner_in = [False for winner in winners]
    winner_idx = [-1 for winner in winners]
    for idx, winner in enumerate(winners):
        if(winner in x.preference):
            winner_in[idx] = True
            winner_idx[idx] = x.preference.index(winner)
            w_found += 1
    for in_, idx in zip(winner_in, winner_idx):
        if(in_ and idx < 2 and idx >=0):
            return False
    if(w_found > 0 and (len(x.preference) - w_found) > 1): 
        return min(winner_idx) 
    if(len(x.preference) > 1):
        return len(x.preference)
    else:
        return False

In [12]:
df[f'consider'] = df.apply(consider_alternative, axis=1)
# df_filtered = df[df['consider'] != False]
df = df.sort_values(by=['n'], ascending=True)

In [13]:
df

,n,preference,consider
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False
1767,0.5,"[2, 4, 1, 5, 6, 9]",6
974,1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False
544,1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2
857,1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False
...,...,...,...
65,4.0,[5],False
49,4.0,"[3, 8, 7]",False
30,7.0,"[3, 4]",2
29,7.0,"[8, 2]",False


In [14]:
df.head(10)

,n,preference,consider
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False
1767,0.5,"[2, 4, 1, 5, 6, 9]",6
974,1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False
544,1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2
857,1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False
270,1.0,"[3, 1, 2, 4, 7, 8, 9, 5, 6]",5
416,1.0,"[7, 8, 3, 6, 2, 1, 9, 5, 4]",False
643,1.0,"[5, 1, 2, 4, 3, 7, 8, 9, 6]",6
1419,1.0,"[8, 9, 4, 3]",False
693,1.0,"[5, 8, 3, 9, 7, 1, 4, 6, 2]",False


In [15]:
df.consider.value_counts()

False    66
2        34
3        22
4        21
5        17
6        13
7         3
8         1
Name: consider, dtype: int64

In [16]:
def create_alternatives(x):
    # return x.preference[:x.consider]
    orderings = []
    if(x.consider != False):
        orderings = [list(item) + x.preference[x.consider:] for item in list(permutations(x.preference[:x.consider]))[1:]]
    # orderings.remove(x.preference[:x.consider])
    return orderings

In [17]:
df.loc[:,'list'] = df.apply(create_alternatives, axis=1)
df.loc[:,'list_counts'] = df.apply(lambda x: len(x.list), axis=1)
df

,n,preference,consider,list,list_counts
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False,[],0
1767,0.5,"[2, 4, 1, 5, 6, 9]",6,"[[2, 4, 1, 5, 9, 6], [2, 4, 1, 6, 5, 9], [2, 4...",719
974,1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False,[],0
544,1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2,"[[5, 2, 8, 7, 1, 3, 9, 6, 4]]",1
857,1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False,[],0
...,...,...,...,...,...
65,4.0,[5],False,[],0
49,4.0,"[3, 8, 7]",False,[],0
30,7.0,"[3, 4]",2,"[[4, 3]]",1
29,7.0,"[8, 2]",False,[],0


In [18]:
# Original counts
# {'1': 246, '2': 453, '3': 412, '4': 384, '5': 352, '6': 27, '7': 94, '8': 463, '9': 37, '10': 0, '11': 0}

In [19]:
winners = STV(df, alternatives)
winners

{'1': 26, '2': 33, '3': 56, '4': 21, '5': 27, '6': 6, '7': 9, '8': 42, '9': 2, '10': 0, '11': 0}
removing alternatives ['10', '11']
{'9': 2, '5': 27, '8': 42, '7': 9, '1': 26, '6': 6, '3': 56, '4': 21, '2': 33}
removing alternatives ['9']
{'5': 27, '8': 43, '7': 9, '1': 26, '6': 6, '3': 57, '4': 21, '2': 33}
removing alternatives ['6']
{'5': 27, '8': 45, '7': 10, '1': 27, '3': 58, '4': 22, '2': 33}
removing alternatives ['7']
{'5': 29, '8': 47, '1': 28, '3': 60, '4': 24, '2': 34}
removing alternatives ['4']
{'5': 31, '8': 51, '1': 32, '3': 63, '2': 45}
removing alternatives ['5']
{'1': 40, '2': 48, '3': 67, '8': 63}
removing alternatives ['1']
{'2': 63, '3': 75, '8': 78}
removing alternatives ['2']
{'3': 94, '8': 101}
removing alternatives ['3']
Reached a decision


['8']

In [20]:
df

,n,preference,consider,list,list_counts
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False,[],0
1767,0.5,"[2, 4, 1, 5, 6, 9]",6,"[[2, 4, 1, 5, 9, 6], [2, 4, 1, 6, 5, 9], [2, 4...",719
974,1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False,[],0
544,1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2,"[[5, 2, 8, 7, 1, 3, 9, 6, 4]]",1
857,1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False,[],0
...,...,...,...,...,...
65,4.0,[5],False,[],0
49,4.0,"[3, 8, 7]",False,[],0
30,7.0,"[3, 4]",2,"[[4, 3]]",1
29,7.0,"[8, 2]",False,[],0


In [21]:
df.loc[:,'preference_s'] = df.loc[:,'preference'].astype(str)

In [22]:
df['preference_s'].unique().shape

(177,)

In [23]:
df = df.set_index('preference_s')
df

,n,preference,consider,list,list_counts
preference_s,,,,,
"['2', '8', '3', '4', '1', '6', '9', '7']",0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False,[],0
"['2', '4', '1', '5', '6', '9']",0.5,"[2, 4, 1, 5, 6, 9]",6,"[[2, 4, 1, 5, 9, 6], [2, 4, 1, 6, 5, 9], [2, 4...",719
"['8', '3', '4', '1', '5', '9', '6', '2']",1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False,[],0
"['2', '5', '8', '7', '1', '3', '9', '6', '4']",1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2,"[[5, 2, 8, 7, 1, 3, 9, 6, 4]]",1
"['8', '3', '7', '1', '6', '9', '5', '2', '4']",1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False,[],0
...,...,...,...,...,...
['5'],4.0,[5],False,[],0
"['3', '8', '7']",4.0,"[3, 8, 7]",False,[],0
"['3', '4']",7.0,"[3, 4]",2,"[[4, 3]]",1


In [24]:
df.iloc[0].list

[]

In [25]:
re.sub('[\]\[\']', '', df.index.to_list()[0]).split(', ')

['2', '8', '3', '4', '1', '6', '9', '7']

In [26]:
df.loc[df.index.to_list()[0],:]

n                                   0.5
preference     [2, 8, 3, 4, 1, 6, 9, 7]
consider                          False
list                                 []
list_counts                           0
Name: ['2', '8', '3', '4', '1', '6', '9', '7'], dtype: object

In [27]:
'asa' in df.index

False

In [28]:
df

,n,preference,consider,list,list_counts
preference_s,,,,,
"['2', '8', '3', '4', '1', '6', '9', '7']",0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",False,[],0
"['2', '4', '1', '5', '6', '9']",0.5,"[2, 4, 1, 5, 6, 9]",6,"[[2, 4, 1, 5, 9, 6], [2, 4, 1, 6, 5, 9], [2, 4...",719
"['8', '3', '4', '1', '5', '9', '6', '2']",1.0,"[8, 3, 4, 1, 5, 9, 6, 2]",False,[],0
"['2', '5', '8', '7', '1', '3', '9', '6', '4']",1.0,"[2, 5, 8, 7, 1, 3, 9, 6, 4]",2,"[[5, 2, 8, 7, 1, 3, 9, 6, 4]]",1
"['8', '3', '7', '1', '6', '9', '5', '2', '4']",1.0,"[8, 3, 7, 1, 6, 9, 5, 2, 4]",False,[],0
...,...,...,...,...,...
['5'],4.0,[5],False,[],0
"['3', '8', '7']",4.0,"[3, 8, 7]",False,[],0
"['3', '4']",7.0,"[3, 4]",2,"[[4, 3]]",1


In [29]:
def get_smallest_change(df, winners, max_n = None):
    for index, row in df.iterrows():
        print(index)
        if(row.consider == False):
            continue
        if(max_n != None and row.consider > max_n):
            continue
        print(row.consider)
        print(row.n)
        for new_combination in row.list:
            new_df = df.copy()
            try:
                if str(new_combination) in new_df.index:
                    new_df.loc[str(new_combination), 'n'] += row.n
                else:
                    new_df.loc[str(new_combination)] = [row.n, new_combination, -1, [], -1]
                new_df.drop(str(new_combination))
                new_winners = STV(new_df, alternatives, verbose=0)
                if(set(winners) != set(new_winners)):
                    print('Manipulation found')
                    return row, new_combination
            except Exception as e:
                print(e)
                return new_df, new_combination
    return 'not found', 'not_found'

info, preferences = get_smallest_change(df, winners, max_n = 3)

['2', '8', '3', '4', '1', '6', '9', '7']
['2', '4', '1', '5', '6', '9']
['8', '3', '4', '1', '5', '9', '6', '2']
['2', '5', '8', '7', '1', '3', '9', '6', '4']
2
1.0
['8', '3', '7', '1', '6', '9', '5', '2', '4']
['3', '1', '2', '4', '7', '8', '9', '5', '6']
['7', '8', '3', '6', '2', '1', '9', '5', '4']
['5', '1', '2', '4', '3', '7', '8', '9', '6']
['8', '9', '4', '3']
['5', '8', '3', '9', '7', '1', '4', '6', '2']
['5', '4', '3', '6', '7', '9', '1', '8', '2']
['2', '5', '4', '9', '8', '10']
['3', '8', '5', '1', '9', '7', '4', '6']
['3', '4', '5', '7', '8']
['1', '2', '4', '5', '7', '8', '9']
['4', '2', '3', '5', '7', '8', '9', '1', '6']
['8', '4', '5', '1', '6', '9', '2', '3', '7']
['4', '3', '1', '5', '8', '7', '6', '9', '2']
['5', '4', '8', '3', '1', '2', '6', '7', '9']
2
1.0
['2', '1', '8', '3', '5', '4']
2
1.0
['3', '5', '8', '1', '4']
2
1.0
['6', '8', '5']
['1', '8', '5', '7', '3', '6', '9', '2', '4']
['4', '2', '5', '3']
['2', '3', '5', '4', '7', '1']
['1', '5', '2', '8', '3', '6']

In [30]:
STV(df, alternatives)

{'1': 26, '2': 33, '3': 56, '4': 21, '5': 27, '6': 6, '7': 9, '8': 42, '9': 2, '10': 0, '11': 0}
removing alternatives ['10', '11']
{'9': 2, '5': 27, '8': 42, '7': 9, '1': 26, '6': 6, '3': 56, '4': 21, '2': 33}
removing alternatives ['9']
{'5': 27, '8': 43, '7': 9, '1': 26, '6': 6, '3': 57, '4': 21, '2': 33}
removing alternatives ['6']
{'5': 27, '8': 45, '7': 10, '1': 27, '3': 58, '4': 22, '2': 33}
removing alternatives ['7']
{'5': 29, '8': 47, '1': 28, '3': 60, '4': 24, '2': 34}
removing alternatives ['4']
{'5': 31, '8': 51, '1': 32, '3': 63, '2': 45}
removing alternatives ['5']
{'1': 40, '2': 48, '3': 67, '8': 63}
removing alternatives ['1']
{'2': 63, '3': 75, '8': 78}
removing alternatives ['2']
{'3': 94, '8': 101}
removing alternatives ['3']
Reached a decision


['8']